In [2]:
import rdkit
from rdkit import Chem
import os
import pandas as pd
from io import StringIO

In [3]:
def parse(f):
    file = open(f,'r')
    content = file.readlines()
    cl_no=0
    saved_linenos=list()
    dic=dict()
    for line in content:
        #print(line)
        cline=line.strip()
        if cline == "## Molecule":
            #save next line that contains the SMILES
            saved_linenos.append(cl_no+1)
            curr_mollineno=cl_no+1
        if cline == "## SMILES":
            #save current line plus 4
            toks_clean=[x.strip() for x in content[cl_no+1].strip().split("|")][2:-1]
            attrel_clean=[x.strip() for x in content[cl_no+3].strip().split("|")][2:-1]
            attabs_clean=[x.strip() for x in content[cl_no+4].strip().split("|")][2:-1]
            dic[content[curr_mollineno].strip()]=[toks_clean,attrel_clean,attabs_clean]
        cl_no+=1
        
    for key, value in dic.items():
        #print(key, value)
        assert(len(value[1])==(len(value[0])))
        #for it in value:
            #print("________",it)
    return dic

In [4]:
def exec_antechamber(inputfile):
    inputfile_noex=os.path.splitext(inputfile)[0]
    print(inputfile_noex)
    os.system(f"antechamber -i {inputfile} -fi mol2 -o {inputfile_noex}_ass.mol2 -fo mol2 -at amber")
    return f"{inputfile_noex}_ass.mol2"

In [5]:
def exec_antechamber_pdb(inputfile):
    inputfile_noex=os.path.splitext(inputfile)[0]
    print("inputfile no extension",inputfile_noex)
    os.system(f"antechamber -i {inputfile} -fi pdb -o {inputfile_noex}_ass.mol2 -fo mol2 -at amber")
    return f"{inputfile_noex}_ass.mol2"

In [6]:
def smilestomol2(smiles,no):
    #obabel [-i<input-type>] <infilename> [-o<output-type>] -O<outfilename> [Options]

    os.system(f'obabel -:"{smiles}" -o mol2 -O ./mols/mol_{no}.mol2 --gen3d')
   # os.system(f'obabel -:"{smiles}" -o pdb -O mol_{no}.pdb')
    return f"./mols/mol_{no}.mol2"

In [7]:
def smilestopdb(smiles,no):
    #obabel [-i<input-type>] <infilename> [-o<output-type>] -O<outfilename> [Options]

    #os.system(f'obabel -:"{smiles}" -o mol2 -O mol_{no}.mol2 --gen3d')
    os.system(f'obabel -:"{smiles}" -o pdb -O ./mols/mol_{no}.pdb')
    return f"./mols/mol_{no}.pdb"

In [8]:
def getatom_ass(mol2):
    #extract lines between @<TRIPOS>ATOM and @<TRIPOS>BOND to get atom asss
    with open(mol2) as infile:
        lines = infile.read().splitlines()
    start = [i for i, line in enumerate(lines) if line.startswith("@<TRIPOS>ATOM")][0]
    end = [i for i, line in enumerate(lines) if line.startswith("@<TRIPOS>BOND")][0]
    extract="\n".join(lines[start+1:end])
    print("\n_______________________extraction \n", extract)
    pddf = pd.read_csv(StringIO(extract), header=None, delimiter=r"\s+")
    #extract 5th column with atom_asss
    atoms_ass_list = pddf.iloc[:,5].tolist()
    atoms_ass_set = set(atoms_ass_list)
    #print("atoms assigned", atoms_ass_list)
    #print()
    #print("assignments set", atoms_ass_set)
    return atoms_ass_list, atoms_ass_set

In [9]:
def clean_SMILES(SMILES_tok, attention_rel, attention_abs):
    SMILES_tok_prep=list()
    attention_relprep=list()
    attention_absprep=list()
    struc_toks=r"()=-:~1234567890#"
    for i in range(len(SMILES_tok)):
        #when it's an H in the SMILES, ignore, cannot deal
        #print(SMILES_tok[i])
        if SMILES_tok[i]!="H" and not SMILES_tok[i].isdigit():
            if any(elem in struc_toks for elem in SMILES_tok[i])==False:
                SMILES_tok_prep.append(SMILES_tok[i])
                attention_relprep.append(attention_rel[i])
                attention_absprep.append(attention_abs[i])
                #print("not H and not number: ", SMILES_tok[i])
            #if any(elem in struc_toks for elem in SMILES_tok[i])==True:
             #   print("H, structure or number: ", SMILES_tok[i])
    print(SMILES_tok_prep)
    return SMILES_tok_prep,attention_relprep,attention_absprep

In [10]:
def clean_acout(ac_out):
    ac_out_noH=list()
    for j in ac_out:
        #save only when it's mot H
        if not j.startswith('H'):
            ac_out_noH.append(j)
            #print(f"-----------------this is not H, this is: {j}")
    #print("before: ", ac_out)
    #print("after: ", ac_out_noH)
    return ac_out_noH

In [11]:
def check_acVStoks(SMILES_tok_prep,ac_out_noH):
    for k in range(len(ac_out_noH)):
        #compare first letter in 
        #print(ac_out_noH[k][0])
        compSMI=SMILES_tok_prep[k][0]
        if SMILES_tok_prep[k][0]=="[":
            compSMI=SMILES_tok_prep[k][1]
        if (ac_out_noH[k][0].upper()!=compSMI.upper()):
            print(f"ac_out and SMILES not the same: {ac_out_noH[k][0]} vs {compSMI}")
        #else:
        #    print(f"letters same: {ac_out_noH[k][0]} vs {compSMI}")

In [12]:
def link_atomstoatoms(ac_out, SMILES_tok, attention_rel, attention_abs):
    dic=dict()
    SMILES_tok_prep=list()
    attention_relprep=list()
    attention_absprep=list()
    print("len smiles tok",len(SMILES_tok))
    print("smiles tok",SMILES_tok)
    print("len_attention rel",len(attention_rel))
    print("len attention abs", len(attention_abs))
    
    assert(len(SMILES_tok)==(len(attention_rel)))
    assert(len(SMILES_tok)==(len(attention_abs)))
    SMILES_tok_prep, attention_relprep, attention_absprep = clean_SMILES(SMILES_tok,attention_rel,attention_abs)
    ac_out_noH = clean_acout(ac_out)
    
    #compare atoms in SMILES_tok with ac_out_noH
   # for h in ac_out_noH:
    if (len(ac_out_noH)!=(len(SMILES_tok_prep))):
        print("!!!!!!!!!!!!!!!!!!ac_out_noH and SMILES_tok_prep do NOT have same lengths: {} vs {}".format(len(ac_out_noH),len(SMILES_tok_prep)))
        print("!!!!!!!!!!!!!!printing all items in ac_out_noH:")
        for g in ac_out_noH:
            print(f"\t {g}")
        print("!!!!!!!!!!!!!!printing all items in SMILES_tok_prep:")
        for h in SMILES_tok_prep:
            print(f"\t {h}")
    #check whether assignment and SMILES tok have same length
    assert(len(ac_out_noH)==(len(SMILES_tok_prep)))
    #cannot check assignment names against toks since not same beginning letter anymore
    #check_acVStoks(SMILES_tok_prep,ac_out_noH)
    
    return SMILES_tok_prep, ac_out_noH, attention_relprep, attention_absprep
    

In [13]:
def write_outputfile(results,outname):
    f=open(f"{outname}.txt",'w') 
    for key, value in results.items():
        f.write(f"{key}\n")
        f.write("{SMILES_token} \t {antechamber_assignment} \t {attention_relative} \t {attention_absolute}\n")   
        for i in range(len(value[0])):
            f.write(f"{results[key][0][i]} \t\t\t {value[1][i]} \t\t\t {value[2][i]} \t\t\t {value[3][i]}\n")
        f.write("\n")
    
    #f = open(outname.txt,"w")
    #f=open(f"{outname}.txt",'w')  
    ##for key, value in results.items():
      #  f.write(f"{key}\n")
      #  f.write("{SMILES_tok_prep[i]} \t {ac_out_noH[i]} \t {attention_relprep[i]} \t {attention_absprep[i]}\n")   
      #  for i in range(len(value[0])):
      #      f.write(f"{results[key][0][i]}\t \t{value[1][i]}\t \t{value[2][i]}\t \t{value[3][i]}\n")
      #  f.write("\n")
    f.close()

## MAIN

In [14]:
#smi = "cc1cc(=O)[nH]c(=S)[nH]1"
#smi_toks = ['C','c','1','c','c','(','=','O',')','[nH]','c','(','=','S',')','[nH]','1']
#smi_atts = ['-0.012','-0.005','0.016','-0.004','-0.009','-0.007','-0.009','-0.003','-0.003','0.001','-0.011','-0.01','-0.004','0.004','0.039','0.024','-0.008']
#smi = "C1=CC=CN=C1"
#smi_toks= ['C','1','=','C','C','=','C','N','=','C','1']
#smi_atts = [1,2,3,4,5,6,7,8,9,10,11]
results=dict()
dic=parse('./markdown_bugfix.md')
no=0
failed_smiToFile=0
failed_fileToac=0
pdb_ass_success=0
for key, value in dic.items():
    no+=1
    smi=key
    smi_toks=value[0]
    smi_relatts=value[1]
    smi_absatts=value[2]
    print()
    print()
    print(f"{smi}--------------------------------------------------------")
    #mol2_file=smilestomol2(smi,no)
    print("smiles to pdb")
    pdb_file=smilestopdb(smi,no)
    if os.path.isfile(pdb_file)==False:
        failed_smiToFile+=1
        print("!!!!!!!!!!!!!!!!!!!!!!FAILURE to generate PDB file!!!!!!!!!!!!!!!!!!!")
        break
    #os.system(f"cat {mol2_file}")
    #mol2_ass = exec_antechamber(mol2_file)
    print(".............executing antechamber")
    mol2_ass = exec_antechamber_pdb(pdb_file)
        
    if os.path.isfile(mol2_ass)==True:
        pdb_ass_success+=1
 #   os.system(f"cat {mol2_ass}")
        atoms_ass_list, atoms_ass_set = getatom_ass(mol2_ass)
        SMILES_tok_prep,ac_out_noH,attention_relprep,attention_absprep=link_atomstoatoms(atoms_ass_list, smi_toks, smi_relatts, smi_absatts)
        results[smi]=[SMILES_tok_prep,ac_out_noH,attention_relprep,attention_absprep]
    else:
        print("!!!!!!!!!!!!!!!!Atom assignment failed!!!!!!!!!!!!!!!!!")
        failed_fileToac+=1
    

#print("{SMILES_tok_prep[i]} \t {ac_out_noH[i]} \t {attention_relprep[i]} \t {attention_absprep[i]}")   
     #   for i in range(len(SMILES_tok_prep)):
      #      print(f"{SMILES_tok_prep[i]} \t {ac_out_noH[i]} \t {attention_relprep[i]} \t {attention_absprep[i]}")
        



c1cc2ccc3cccc4ccc(c1)c2c34--------------------------------------------------------
smiles to pdb
.............executing antechamber
inputfile no extension ./mols/mol_1

Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to amber; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for pdb File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
         It is quite possible that there are unfilled valences.
-- Check Geometry --
      for those bonded   
      for those not bonded   
-- Check Weird Bonds --
   Status: pass
-- Check Number of Units --
   Status: pass
acdoctor mode has completed checking the input file.

Info: The number of rings (1712) exceeded the default (1000).
(1) double check the structure (the connectivity) and/or 
(2) adjust atom valence penalty parameters in APS.DAT

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


(1) double check the structure (the connectivity) and/or 
(2) adjust atom valence penalty parameters in APS.DAT, and/or 
(3) increase PSCUTOFF in define.h and recompile bondtype.c
    (be cautious, using a large value of PSCUTOFF (>100) will 
    significantly increase the computation time).


_______________________extraction 
       1 C            0.0000     0.0000     0.0000 DU         1 UNL       0.000000
      2 C1           0.0000     0.0000     0.0000 CM         1 UNL       0.000000
      3 C2           0.0000     0.0000     0.0000 CZ         1 UNL       0.000000
      4 O            0.0000     0.0000     0.0000 OS         1 UNL       0.000000
len smiles tok 6
smiles tok ['C', 'C', '1', 'C', 'O', '1']
len_attention rel 6
len attention abs 6
['C', 'C', 'C', 'O']


CCN1c2ccccc2N(C)C(=S)c2cccnc21--------------------------------------------------------
smiles to pdb
.............executing antechamber
inputfile no extension ./mols/mol_7

Welcome to antechamber 22.0: molecular input f

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
/home/inken/anaconda3/envs/AmberTools/bin/wrapped_progs/antechamber: Fatal Error!
This molecule may have more than one unit.
       antechamber can only handle one unit.  If the input is a single unit
       then the connectivity is wrong and the geometry may be bad.
       Please convert your molecule to a mol2 file via:
       antechamber -j 5 -at sybyl -dr no 
       And then check your molecule with a visualization program;
       manually add missing bonds or delete unwanted bonds as appropriate.
1 molecule converted
1 molecule converted


len smiles tok 30
smiles tok ['c', '*1*', 'c', 'c', 'c', '(', '-', 'c', '*2*', 'c', 'c', 'c', '(', '-', 'c', '3', 'c', 'c', 'c', 'c', 'c', '3', ')', 'c', 'c', '*2*', ')', 'c', 'c', '1']
len_attention rel 30
len attention abs 30
['c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c']


CC12CC(=O)C3C(CCC4=CC(=O)CCC43C)C1CCC2=O--------------------------------------------------------
smiles to pdb
.............executing antechamber
inputfile no extension ./mols/mol_15

Welcome to antechamber 22.0: molecular input file processor.

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to amber; the options available to the -at flag are
      gaff, gaff2, amber, bcc, and sybyl.

-- Check Format for pdb File --
   Status: pass
-- Check Unusual Elements --
   Status: pass
-- Check Open Valences --
         It is quite possible that there are unfilled valences.
-- Check Geometry --
      for those bonded   
      f

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


In [15]:
print(f"failed translation SMILES to pdb file: {failed_smiToFile} out of {no} times")
print(f"successful pdb assignment with antechamber: {pdb_ass_success} out of {no} times")
#print(f"Failed to translate {failed_smiToFile} SMILES to file")
#print(f"Failed to translate {failed_mol2Toac} mol2 to antechamber atom assignment")

failed translation SMILES to pdb file: 0 out of 20 times
successful pdb assignment with antechamber: 20 out of 20 times


In [16]:
for key, value in results.items():
    print(key)
    print("{SMILES_token} \t {antechamber_assignment} \t {attention_relative} \t {attention_absolute}")   
    for i in range(len(value[0])):
        print(f"{results[key][0][i]} \t \t \t {value[1][i]} \t \t \t {value[2][i]} \t \t \t \t {value[3][i]} \t")
    print()

c1cc2ccc3cccc4ccc(c1)c2c34
{SMILES_token} 	 {antechamber_assignment} 	 {attention_relative} 	 {attention_absolute}
c 	 	 	 CD 	 	 	 -0.006 	 	 	 	 0.032 	
c 	 	 	 CM 	 	 	 -0.002 	 	 	 	 0.036 	
c 	 	 	 CT 	 	 	 -0.011 	 	 	 	 0.027 	
c 	 	 	 CM 	 	 	 -0.005 	 	 	 	 0.033 	
c 	 	 	 CM 	 	 	 -0.014 	 	 	 	 0.025 	
c 	 	 	 CT 	 	 	 -0.008 	 	 	 	 0.031 	
c 	 	 	 CM 	 	 	 -0.012 	 	 	 	 0.027 	
c 	 	 	 CD 	 	 	 -0.011 	 	 	 	 0.027 	
c 	 	 	 CM 	 	 	 -0.008 	 	 	 	 0.03 	
c 	 	 	 CT 	 	 	 -0.015 	 	 	 	 0.023 	
c 	 	 	 CM 	 	 	 -0.012 	 	 	 	 0.026 	
c 	 	 	 CM 	 	 	 -0.012 	 	 	 	 0.027 	
c 	 	 	 CT 	 	 	 -0.013 	 	 	 	 0.025 	
c 	 	 	 CM 	 	 	 -0.014 	 	 	 	 0.024 	
c 	 	 	 CT 	 	 	 -0.016 	 	 	 	 0.023 	
c 	 	 	 CT 	 	 	 -0.012 	 	 	 	 0.027 	

Cc1cc(=O)[nH]c(=S)[nH]1
{SMILES_token} 	 {antechamber_assignment} 	 {attention_relative} 	 {attention_absolute}
C 	 	 	 DU 	 	 	 -0.034 	 	 	 	 0.025 	
c 	 	 	 CT 	 	 	 -0.015 	 	 	 	 0.044 	
c 	 	 	 CM 	 	 	 -0.032 	 	 	 	 0.027 	
c 	 	 	 CT 	 

In [17]:
write_outputfile(results,"./mols/results_atomassignment_ac_new")

In [18]:
mol12="CN1C(C(=O)Nc2ccccn2)=C(O)c2ccccc2S1(=O)=O"
smilestomol2(mol12,12)


1 molecule converted


'./mols/mol_12.mol2'